In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
from os.path import join
import re
from os import makedirs
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
from scipy.stats import pearsonr
from IPython.display import display
from skopt.plots import plot_objective
import seaborn as sns
from time import time
import skopt
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import xgboost as xgb

rng_seed = 399
np.random.seed(rng_seed)
import persim
import joblib

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_squared_error,
    f1_score,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.linear_model import Lasso, LassoCV, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], "GPU")
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != "GPU"
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

from tensorflow.keras.activations import relu
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l1
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# Directory constants
root_code_dir = ".."
output_dir = join(root_code_dir, "output")
word2vec_training_dir = join(output_dir, "word2vec_training")
word2vec_ann_indices_dir = join(output_dir, "word2vec_ann_indices")
word2vec_cluster_analysis_dir = join(output_dir, "word2vec_cluster_analysis")
output_plots_dir = join("output_plots")
makedirs(output_plots_dir, exist_ok=True)

# Extend sys path for importing custom Python files
import sys

sys.path.append(root_code_dir)

from topological_data_analysis.topological_polysemy import tps
from word_embeddings.word2vec import load_model_training_output
from analysis_of_embeddings.estimate_num_meanings_supervised import (
    create_classification_labels,
    evaluate_regression_model,
    evaluate_classification_model,
    create_feature_importance_df,
    visualize_feature_importances,
)
from vis_utils import configure_plotting_for_thesis

configure_plotting_for_thesis()

## Prepare data

In [ ]:
def format_feature_name_human_readable(feature_name: str) -> str:
    """
    TODO: Docs
    """
    alg_names = ["tps", "gad", "estimated_id"]
    human_readable_regexes = [
        r"X_tps_(\d+)(_pd_(?:max|avg|std)|)",
        r"X_gad_knn_(\d+)_(\d+)_(P_man|P_bnd|P_int)",
        r"X_estimated_id_(.+)_(\d+)",
    ]
    for alg_name, human_readable_re in zip(alg_names, human_readable_regexes):
        re_match = re.match(human_readable_re, feature_name)
        if re_match is None:
            continue
        re_groups = re_match.groups()
        if alg_name == "tps":
            tps_n = re_groups[0]
            if re_groups[1] is None:
                return fr"TPS$_{tps_n}"
            else:
                tps_pd_type = re_groups[1]
                return fr"TPS{tps_pd_type}_{tps_n}"
        elif alg_name == "gad":
            inner_annulus_knn, outer_annulus_knn, P_cat = re_groups
            P_cat_human = {
                "P_man": "manifold",
                "P_bnd": "boundary",
                "P_int": "singular",
            }
            return fr"GAD_{P_cat_human[P_cat]}_{inner_annulus_knn}_{outer_annulus_knn}"
        elif alg_name == "estimated_id":
            id_estimator_name, num_neighbours = re_groups
            id_estimator_human = {
                "lpca": "LPCA",
                "knn": "KNN",
                "twonn": "TWO-NN",
                "mle": "MLE",
                "tle": "TLE",
            }
            return fr"ID_{id_estimator_human[id_estimator_name]}_{num_neighbours}"

In [ ]:
word_meaning_train_data = pd.read_csv("data/word_meaning_train_data.csv")
word_meaning_test_data = pd.read_csv("data/word_meaning_test_data.csv")
word_meaning_semeval_test_data = pd.read_csv("data/word_meaning_semeval_test_data.csv")
word_meaning_data_cols = word_meaning_train_data.columns.values
word_meaning_data_feature_cols = np.array(
    [col for col in word_meaning_data_cols if col.startswith("X_")]
)
word_meaning_data_feature_cols_human_readable = np.array(
    [format_feature_name_human_readable(col) for col in word_meaning_data_feature_cols]
)

In [ ]:
print("Train")
word_meaning_train_data

In [ ]:
plt.hist(word_meaning_train_data["y"], bins=word_meaning_train_data["y"].max())
plt.xlabel("Label y")
plt.ylabel("Count")
plt.show()

In [ ]:
print("Test")
word_meaning_test_data

In [ ]:
plt.hist(word_meaning_test_data["y"], bins=word_meaning_test_data["y"].max())
plt.xlabel("Label y")
plt.ylabel("Count")
plt.show()

In [ ]:
# Split into X and y
data_scaler = StandardScaler()
data_scaler.fit(word_meaning_train_data[word_meaning_data_feature_cols].values)
X_train = data_scaler.transform(word_meaning_train_data[word_meaning_data_feature_cols].values)
X_test = data_scaler.transform(word_meaning_test_data[word_meaning_data_feature_cols].values)
X_test_semeval = data_scaler.transform(word_meaning_semeval_test_data[word_meaning_data_feature_cols].values)
y_train = word_meaning_train_data["y"].values
y_test = word_meaning_test_data["y"].values
y_test_semeval = word_meaning_semeval_test_data["y"].values

In [ ]:
# Create multi-class labels
max_y_multi = np.quantile(y_train, q=0.9)
y_train_binary_classes = create_classification_labels(labels=y_train, max_label=1)
y_train_multi_class = create_classification_labels(
    labels=y_train, max_label=max_y_multi
)
y_test_binary_classes = create_classification_labels(labels=y_test, max_label=1)
y_test_multi_class = create_classification_labels(labels=y_test, max_label=max_y_multi)
y_test_semeval_binary_classes = create_classification_labels(
    labels=y_test_semeval, max_label=1
)
y_test_semeval_multi_class = create_classification_labels(
    labels=y_test_semeval, max_label=max_y_multi
)
labels_str = [
    str(label + 1) if i < 4 else "gt_or_eq_5"
    for i, label in enumerate(np.unique(y_train_multi_class))
]

In [ ]:
# Load output from training word2vec
w2v_training_output = load_model_training_output(
    model_training_output_dir=join(
        word2vec_training_dir, "word2vec_enwiki_jan_2021_word2phrase"
    ),
    model_name="word2vec",
    dataset_name="enwiki",
    return_normalized_embeddings=True,
)
last_embedding_weights_normalized = w2v_training_output[
    "last_embedding_weights_normalized"
]
words = w2v_training_output["words"]
word_to_int = w2v_training_output["word_to_int"]
word_counts = w2v_training_output["word_counts"]

In [ ]:
# Load SemEval-2010 task 14 words
semeval_2010_14_word_senses = joblib.load(
    join(
        "..", "topological_data_analysis", "data", "semeval_2010_14_word_senses.joblib"
    )
)
semeval_target_words = np.array(list(semeval_2010_14_word_senses["all"].keys()))
semeval_target_words_in_vocab_filter = [
    i for i, word in enumerate(semeval_target_words) if word in word_to_int
]
semeval_target_words_in_vocab = semeval_target_words[
    semeval_target_words_in_vocab_filter
]
semeval_gs_clusters = np.array(list(semeval_2010_14_word_senses["all"].values()))
semeval_gs_clusters_in_vocab = semeval_gs_clusters[semeval_target_words_in_vocab_filter]

num_semeval_words = len(semeval_target_words_in_vocab)

## Evaluate modeling results

In [ ]:
# Constants
estimate_num_meanings_supervised_dir = join("data", "estimate_num_meanings_supervised")

### LASSO / Logistic regression

#### LASSO

In [ ]:
# Load results
lasso_reg = joblib.load(join(estimate_num_meanings_supervised_dir, "lasso_reg.joblib"))

In [ ]:
print(f"Selected alpha: {lasso_reg.alpha_:.16f}")

In [ ]:
# LASSO regression
evaluate_regression_model(
    model=lasso_reg,
    test_sets=[
        (
            X_train,
            y_train,
            "Train",
            "Predicted number of word meanings",
            "Synsets in WordNet",
        ),
        (
            X_test,
            y_test,
            "Test",
            "Predicted number of word meanings",
            "Synsets in WordNet",
        ),
        (
            X_test_semeval,
            y_test_semeval,
            "SemEval test",
            "Predicted number of word meanings",
            "SemEval gold standard",
        ),
    ],
    show_plot=False,
    use_rasterization=True,
)

# Plot/save
save_to_pgf = True
plt.tight_layout()
if save_to_pgf:
    plt.savefig(
        join(
            output_plots_dir,
            "wme-enwiki-correlation-result.pdf",
        ),
        backend="pgf",
    )
else:
    plt.show()

In [ ]:
# Visualize top 10 feature importances
_, ax = plt.subplots(figsize=(10, 5))

# Sort coefficient by absolute value
lasso_reg_coef_abs_sorted_indces = np.argsort(abs(lasso_reg.coef_))[::-1]
top_n_importances = 10
top_n_importances_indices = lasso_reg_coef_abs_sorted_indces[:top_n_importances]

# Plot horizontal barplot
y_pos = np.arange(top_n_importances)
ax.barh(y=y_pos, width=lasso_reg.coef_[top_n_importances_indices], color="b")
ax.set_yticks(y_pos)
ax.set_yticklabels(
    word_meaning_data_feature_cols_human_readable[top_n_importances_indices]
)
ax.invert_yaxis()
ax.set_xlabel("Feature importance")

# Plot/save
save_to_pgf = True
plt.tight_layout()
if save_to_pgf:
    plt.savefig(
        join(
            output_plots_dir,
            "wme-enwiki-top-10-feature-importances.pdf",
        ),
        backend="pgf",
    )
else:
    plt.show()

In [ ]:
# Visualize top 10 feature importances
_, axes = plt.subplots(ncols=3, figsize=(15, 5))
ax_chars = "abc"

top_n_importances = 10
feature_alg_names = ["TPS", "GAD", "ID estimator"]
feature_alg_names_start = ["X_tps", "X_gad", "X_estimated_id"]
for ax, ax_char, alg_name, alg_names_start in zip(
    axes, ax_chars, feature_alg_names, feature_alg_names_start
):
    # Filter algorithm columns
    alg_filter = [
        i
        for i, feature_col in enumerate(word_meaning_data_feature_cols)
        if feature_col.startswith(alg_names_start)
    ]
    alg_coeffs = lasso_reg.coef_[alg_filter]

    # Sort coefficient by absolute value
    lasso_reg_coef_abs_sorted_indces = np.argsort(abs(alg_coeffs))[::-1]
    top_n_importances_indices = lasso_reg_coef_abs_sorted_indces[:top_n_importances]

    # Plot horizontal barplot
    y_pos = np.arange(top_n_importances)
    ax.barh(y=y_pos, width=alg_coeffs[top_n_importances_indices], color="b")
    ax.set_yticks(y_pos)
    ax.set_yticklabels(
        word_meaning_data_feature_cols_human_readable[alg_filter][
            top_n_importances_indices
        ]
    )
    ax.invert_yaxis()
    ax.set_xlabel("Feature importance")
    ax.set_title(f"({ax_char}) {alg_name} features")

# Plot/save
save_to_pgf = True
plt.tight_layout()
if save_to_pgf:
    plt.savefig(
        join(
            output_plots_dir,
            "wme-enwiki-top-10-feature-importances-tps-gad-estimated-ids.pdf",
        ),
        backend="pgf",
    )
else:
    plt.show()

In [ ]:
visualize_feature_importances(
    feature_importances=create_feature_importance_df(
        feature_names=word_meaning_data_feature_cols,
        feature_importances=np.abs(lasso_reg.coef_),
    )
)

In [ ]:
print(f"Number of zero features: {sum(lasso_reg.coef_ == 0)}")

#### Logistic regression with L1 penalty

In [ ]:
# Load results
binary_logistic_reg = joblib.load(
    join(estimate_num_meanings_supervised_dir, "binary_logistic_reg.joblib")
)

In [ ]:
print(f"Selected alpha: {(1 / binary_logistic_reg.C_[0]):.16f}")

In [ ]:
# Binary classification
evaluate_classification_model(
    model=binary_logistic_reg,
    test_sets=[
        (
            X_train,
            y_train_binary_classes,
            "Train",
            "Predicted number of word meanings",
            "Synsets in WordNet",
        ),
        (
            X_test,
            y_test_binary_classes,
            "Test",
            "Predicted number of word meanings",
            "Synsets in WordNet",
        ),
    ],
    cm_ticklabels=["1 word meaning", ">1 word meanings"],
    show_plot=False,
)

# Plot/save
save_to_pgf = True
plt.tight_layout()
if save_to_pgf:
    plt.savefig(
        join(
            output_plots_dir,
            "bwme-enwiki-confusion-matrices.pdf",
        ),
        backend="pgf",
    )
else:
    plt.show()

In [ ]:
# Visualize top 10 feature importances
_, ax = plt.subplots(figsize=(10, 5))

# Sort coefficient by absolute value
binary_log_reg_coef_abs_sorted_indces = np.argsort(abs(binary_logistic_reg.coef_[0]))[
    ::-1
]
top_n_importances = 10
top_n_importances_indices = binary_log_reg_coef_abs_sorted_indces[:top_n_importances]

# Plot horizontal barplot
y_pos = np.arange(top_n_importances)
ax.barh(
    y=y_pos, width=binary_logistic_reg.coef_[0][top_n_importances_indices], color="b"
)
ax.set_yticks(y_pos)
ax.set_yticklabels(
    word_meaning_data_feature_cols_human_readable[top_n_importances_indices]
)
ax.invert_yaxis()
ax.set_xlabel("Feature importance")

# Plot/save
save_to_pgf = True
plt.tight_layout()
if save_to_pgf:
    plt.savefig(
        join(
            output_plots_dir,
            "bwme-enwiki-top-10-feature-importances.pdf",
        ),
        backend="pgf",
    )
else:
    plt.show()

In [ ]:
# Visualize top 10 feature importances
_, axes = plt.subplots(ncols=3, figsize=(15, 5))
ax_chars = "abc"

top_n_importances = 10
feature_alg_names = ["TPS", "GAD", "ID estimator"]
feature_alg_names_start = ["X_tps", "X_gad", "X_estimated_id"]
for ax, ax_char, alg_name, alg_names_start in zip(
    axes, ax_chars, feature_alg_names, feature_alg_names_start
):
    # Filter algorithm columns
    alg_filter = [
        i
        for i, feature_col in enumerate(word_meaning_data_feature_cols)
        if feature_col.startswith(alg_names_start)
    ]
    alg_coeffs = binary_logistic_reg.coef_[0][alg_filter]

    # Sort coefficient by absolute value
    binary_log_reg_coef_abs_sorted_indces = np.argsort(abs(alg_coeffs))[::-1]
    top_n_importances_indices = binary_log_reg_coef_abs_sorted_indces[
        :top_n_importances
    ]

    # Plot horizontal barplot
    y_pos = np.arange(top_n_importances)
    ax.barh(y=y_pos, width=alg_coeffs[top_n_importances_indices], color="b")
    ax.set_yticks(y_pos)
    ax.set_yticklabels(
        word_meaning_data_feature_cols_human_readable[alg_filter][
            top_n_importances_indices
        ]
    )
    ax.invert_yaxis()
    ax.set_xlabel("Feature importance")
    ax.set_title(f"({ax_char}) {alg_name} features")

# Plot/save
save_to_pgf = True
plt.tight_layout()
if save_to_pgf:
    plt.savefig(
        join(
            output_plots_dir,
            "bwme-enwiki-top-10-feature-importances-tps-gad-estimated-ids.pdf",
        ),
        backend="pgf",
    )
else:
    plt.show()

In [ ]:
visualize_feature_importances(
    feature_importances=create_feature_importance_df(
        feature_names=word_meaning_data_feature_cols,
        feature_importances=np.abs(binary_logistic_reg.coef_[0]),
    )
)

In [ ]:
print(f"Number of zero features: {sum(binary_logistic_reg.coef_[0] == 0)}")

# Old cells below

#### Multinomial logistic regression with L1 penalty

In [ ]:
alphas = 1 / multi_class_logistic_reg.C_
print(f"Selected alphas: {alphas}")

In [ ]:
df_dict = {}
for label_str, coeffs in zip(labels_str, multi_class_logistic_reg.coef_):
    sorted_feature_weights_indices = np.argsort(np.abs(coeffs))[::-1]
    df_dict[f"feature_{label_str}"] = word_meaning_data_feature_cols[
        sorted_feature_weights_indices
    ]
    df_dict[f"weight_{label_str}"] = coeffs[sorted_feature_weights_indices]

sorted_features_df = pd.DataFrame(df_dict)
visualize_feature_importances(feature_importances=sorted_features_df)

In [ ]:
# Multi classification
evaluate_classification_model(
    model=multi_class_logistic_reg,
    test_sets=[
        (
            X_train,
            y_train_multi_class,
            "Train",
            "Pred number of synsets",
            "Synsets in Wordnet",
        ),
        (
            X_test,
            y_test_multi_class,
            "Test",
            "Pred number of synsets",
            "Synsets in Wordnet",
        ),
    ],
    cm_ticklabels=labels_str,
)

### XGBoost regression and classification

In [ ]:
# Load results
xgb_reg = joblib.load(join(estimate_num_meanings_supervised_dir, "xgb_reg.joblib"))
xgb_reg_model = xgb_reg.best_estimator_
xgb_binary_classification = joblib.load(
    join(estimate_num_meanings_supervised_dir, "xgb_binary_classification.joblib")
)
xgb_binary_classification_model = xgb_binary_classification.best_estimator_
# xgb_multi_classification = joblib.load(join(estimate_num_meanings_supervised_dir, "xgb_multi_classification.joblib"))
# xgb_multi_classification_model = xgb_multi_classification.best_estimator_

#### XGBoost regression

In [ ]:
# Visualize bayesian optimization results
plot_objective(xgb_reg.optimizer_results_[0])

In [ ]:
visualize_feature_importances(
    feature_importances=create_feature_importance_df(
        feature_names=word_meaning_data_feature_cols,
        feature_importances=xgb_reg_model.feature_importances_,
    )
)

In [ ]:
evaluate_regression_model(
    model=xgb_reg_model,
    test_sets=[
        (X_train, y_train, "Train", "Pred number of synsets", "Synsets in Wordnet"),
        (X_test, y_test, "Test", "Pred number of synsets", "Synsets in Wordnet"),
        (
            X_test_semeval,
            y_test_semeval,
            "SemEval GS",
            "Pred number of meanings",
            "Clusters in GS",
        ),
    ],
)

#### XGBoost binary classification

In [ ]:
# Visualize bayesian optimization results
plot_objective(xgb_binary_classification.optimizer_results_[0])

In [ ]:
visualize_feature_importances(
    feature_importances=create_feature_importance_df(
        feature_names=word_meaning_data_feature_cols,
        feature_importances=xgb_binary_classification_model.feature_importances_,
    )
)

In [ ]:
# Binary classification
evaluate_classification_model(
    model=xgb_binary_classification_model,
    test_sets=[
        (
            X_train,
            y_train_binary_classes,
            "Train",
            "Pred number of synsets",
            "Synsets in Wordnet",
        ),
        (
            X_test,
            y_test_binary_classes,
            "Test",
            "Pred number of synsets",
            "Synsets in Wordnet",
        ),
    ],
    cm_ticklabels=["1 meaning", "2 or more meanings"],
)

#### XGBoost multiclassification

In [ ]:
# Visualize bayesian optimization results
plot_objective(xgb_multi_classification.optimizer_results_[0])

In [ ]:
visualize_feature_importances(
    feature_importances=create_feature_importance_df(
        feature_names=word_meaning_data_feature_cols,
        feature_importances=xgb_multi_classification_model.feature_importances_,
    )
)


In [ ]:
# Multi classification
evaluate_classification_model(
    model=xgb_multi_classification_model,
    test_sets=[
        (
            X_train,
            y_train_multi_class,
            "Train",
            "Pred number of synsets",
            "Synsets in Wordnet",
        ),
        (
            X_test,
            y_test_multi_class,
            "Test",
            "Pred number of synsets",
            "Synsets in Wordnet",
        ),
    ],
    cm_ticklabels=labels_str,
)